In [16]:
import gc
import time
import numpy as np
import pandas as pd
import warnings

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold

import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from project_utils import kd_utils
from importlib import reload
from multiprocessing import Pool


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
from sklearn.pipeline import make_pipeline, make_union, Pipeline
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [17]:
train=pd.read_csv('../input/train.csv', sep='\t')
test=pd.read_csv('../input/test.csv', sep='\t')
train['question_utc']=pd.to_datetime(train['question_utc'], unit='s')
test['question_utc']=pd.to_datetime(test['question_utc'], unit='s')
train['answer_utc']=pd.to_datetime(train['answer_utc'], unit='s')
test['answer_utc']=pd.to_datetime(test['answer_utc'], unit='s')

In [18]:
train = pd.concat([train, kd_utils.load_data('../feature_data/train_fs1.pkl')], axis=1)
test = pd.concat([test, kd_utils.load_data('../feature_data/test_fs1.pkl')], axis=1)

In [19]:
gc.collect()

USECOLS = ['a_dow', 'a_hour', 'timediff_a-q', 'a_len', 'a_count_words', 'a_count_unq_words']

In [15]:
name='train'
'../feature_data/'+name+'_fs2'

'../feature_data/train_fs2'

In [20]:
def nunique(x):
    return len(set(x))


def make_features(p):
    if p == 0:
        df = train
        name = 'train'
    else:
        df = test
        name = 'test'

    init_col = df.columns.tolist()

    gr = df.groupby('question_id')

    for c in USECOLS:
        print(name, c)
        df[f'{c}_min'] = gr[c].transform(np.min)
        df[f'{c}_max'] = gr[c].transform(np.max)
        df[f'{c}_max-min'] = df[f'{c}_max'] - df[f'{c}_min']
        df[f'{c}_mean'] = gr[c].transform(np.mean)
        df[f'{c}_std'] = gr[c].transform(np.std)
        df[f'{c}_nunique'] = gr[c].transform(nunique)

    df.drop(init_col, axis=1, inplace=True)
    kd_utils.pickle_data('../feature_data/'+name+'_fs2', df)


# =============================================================================
#
# =============================================================================

pool = Pool(2)
pool.map(make_features, [0, 1])
pool.close()


test a_dow
train a_dow
test a_hour
train a_hour
test timediff_a-q
train timediff_a-q
test a_len
train a_len
test a_count_words
train a_count_words
test a_count_unq_words
train a_count_unq_words


In [13]:
train.columns

Index(['id', 'question_id', 'subreddit', 'question_utc', 'question_text',
       'question_score', 'answer_utc', 'answer_text', 'answer_score', 'q_dow',
       'q_hour', 'a_dow', 'a_hour', 'timediff_a-q', 'q_len', 'a_len',
       'q_count_words', 'a_count_words', 'q_count_unq_words',
       'a_count_unq_words'],
      dtype='object')